# 02. Feature Engineering для VK EdTech ML Challenge

В этом ноутбуке:
* строим признаки кампаний на основе users, history и validate;
* добавляем кластеры по демографии пользователей;
* считаем rolling-статистики по истории показов;
* собираем итоговый датафрейм `features_df` (одна строка = одна кампания).


# ЭТАП 1. Импорт библиотек и загрузка данных

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import random

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# Визуализации по желанию
import matplotlib.pyplot as plt
import seaborn as sns

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Путь к данным тот же, что и в 01_eda.ipynb
FOLDER = "/content/drive/MyDrive/VK_Project_v2"

users = pd.read_csv(os.path.join(FOLDER, "users.tsv"), sep="\t")
history = pd.read_csv(os.path.join(FOLDER, "history.tsv"), sep="\t")
validate = pd.read_csv(os.path.join(FOLDER, "validate.tsv"), sep="\t")
validate_answers = pd.read_csv(os.path.join(FOLDER, "validate_answers.tsv"), sep="\t")

print("users.shape:", users.shape)
print("history.shape:", history.shape)
print("validate.shape:", validate.shape)
print("validate_answers.shape:", validate_answers.shape)

users.shape: (27769, 4)
history.shape: (1147857, 4)
validate.shape: (1008, 6)
validate_answers.shape: (1008, 3)


# ЭТАП 2. Подготовка кампаний: разворачиваем user_ids сразу в int

In [3]:
validate_expanded = validate.copy()
validate_expanded["user_ids_list"] = validate_expanded["user_ids"].str.split(",")

# "взрыв" по пользователям: каждая строка кампании -> много строк (по user_id)
validate_exploded = validate_expanded.explode("user_ids_list").copy()

# приводим к int уже после explode
validate_exploded["user_ids_list"] = validate_exploded["user_ids_list"].astype(int)
validate_exploded = validate_exploded.rename(columns={"user_ids_list": "user_id"})

print("validate_exploded.shape:", validate_exploded.shape)
display(validate_exploded.head())


validate_exploded.shape: (1098808, 7)


,cpm,hour_start,hour_end,publishers,audience_size,user_ids,user_id
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1...",12
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1...",44
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1...",46
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1...",50
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1...",58


# ЭТАП 3. Демография кампании: средний пол, возраст, число городов

In [4]:
# join кампаний с users по user_id
campaign_users = validate_exploded.merge(users, on="user_id", how="left")

# группируем по индексу кампании (строке validate)
demography_agg = campaign_users.groupby(campaign_users.index).agg(
    sex_mean=("sex", "mean"),
    age_mean=("age", "mean"),
    city_nunique=("city_id", "nunique"),
)

demography_agg.head()

,sex_mean,age_mean,city_nunique
0,2.0,26.0,1
1,2.0,33.0,1
2,2.0,20.0,1
3,1.0,41.0,1
4,1.0,36.0,1


# ЭТАП 4. Кластеры кампаний по демографии

In [5]:
demography_features = demography_agg[["sex_mean", "age_mean"]].fillna(0)

kmeans = KMeans(
    n_clusters=4,      # как в исходном решении (можно будет тюнить)
    random_state=SEED,
    n_init="auto",
)
campaign_clusters = kmeans.fit_predict(demography_features)

demography_agg["campaign_cluster"] = campaign_clusters
demography_agg.head()

,sex_mean,age_mean,city_nunique,campaign_cluster
0,2.0,26.0,1,1
1,2.0,33.0,1,1
2,2.0,20.0,1,2
3,1.0,41.0,1,3
4,1.0,36.0,1,1


# ЭТАП 5. Rolling-статистики по истории показов

In [6]:
history_roll = history.copy()

# hour уже целое число, сортируем по времени
history_roll = history_roll.sort_values("hour")

# rolling окно 24 часа по cpm (по всей истории, без группировки)
# при желании можно делать отдельный rolling по publisher или user_id
history_roll["roll_cpm_mean"] = history_roll["cpm"].rolling(window=24, min_periods=1).mean()
history_roll["roll_cpm_median"] = history_roll["cpm"].rolling(window=24, min_periods=1).median()
history_roll["roll_cpm_std"] = history_roll["cpm"].rolling(window=24, min_periods=1).std().fillna(0)

history_roll.head()

,hour,cpm,publisher,user_id,roll_cpm_mean,roll_cpm_median,roll_cpm_std
49,3,163.49,1,15004,163.490000,163.490,0.000000
1126510,3,34.55,1,15015,99.020000,99.020,91.174348
1126457,3,174.82,1,20794,124.286667,163.490,77.920435
5500,3,255.00,3,4127,156.965000,169.155,91.209777
9153,3,151.05,1,12942,155.782000,163.490,79.034265


# ЭТАП 6. Связь history и кампаний: фильтрация по пользователям и окну часов

In [7]:
# ЭТАП 6. Связь history и кампаний: фильтрация по пользователям и окну часов

history_roll["user_id"] = history_roll["user_id"].astype(int)

# join по user_id сразу с validate_exploded
hist_with_campaign = history_roll.merge(
    validate_exploded[["hour_start", "hour_end", "audience_size", "user_id"]],
    on="user_id",
    how="inner",
)

# фильтруем по временному окну кампании
mask_time = (hist_with_campaign["hour"] >= hist_with_campaign["hour_start"]) & (
    hist_with_campaign["hour"] <= hist_with_campaign["hour_end"]
)
hist_with_campaign = hist_with_campaign[mask_time].copy()

print("hist_with_campaign.shape:", hist_with_campaign.shape)
display(hist_with_campaign.head())

hist_with_campaign.shape: (3663838, 10)


,hour,cpm,publisher,user_id,roll_cpm_mean,roll_cpm_median,roll_cpm_std,hour_start,hour_end,audience_size
23990555,747,30.00,1,5408,198.223750,150.000,158.139874,747,955,1080
23991831,747,30.58,9,18443,336.472500,163.500,540.234573,747,806,1000
23992252,747,117.50,2,11848,241.019583,141.000,492.320723,747,806,1000
23994012,747,150.55,1,26620,245.142500,197.645,209.900873,747,806,1000
23994752,747,225.72,1,3151,295.968333,227.860,285.061738,747,955,1080


# ЭТАП 7. Агрегации по пользователям в рамках кампании

In [8]:
# Группируем по индексу кампании (как раньше по campaign_users.index)
# Здесь предполагаем, что индекс validate_expanded совпадает с исходным индексом validate
hist_with_campaign["campaign_idx"] = hist_with_campaign["hour_start"].map(
    dict(zip(validate_expanded["hour_start"], validate_expanded.index))
)

user_agg = (
    hist_with_campaign.groupby("campaign_idx")
    .agg(
        in_history_frac=("user_id", lambda x: x.nunique() / validate_expanded.loc[x.name, "audience_size"]),
        n_shows_mean=("user_id", "count"),
        n_unique_pub_mean=("publisher", "nunique"),
        mean_cpm_mean=("cpm", "mean"),
        median_cpm_mean=("cpm", "median"),
        quantile_cpm_10=("cpm", lambda x: x.quantile(0.10)),
        quantile_cpm_90=("cpm", lambda x: x.quantile(0.90)),
        roll_cpm_mean=("roll_cpm_mean", "mean"),
        roll_cpm_median=("roll_cpm_median", "median"),
        roll_cpm_std=("roll_cpm_std", "mean"),
    )
)

user_agg.head()

/tmp/ipython-input-584833544.py:9: FutureWarning: Pinning the groupby key to each group in SeriesGroupBy.agg is deprecated, and cases that relied on it will raise in a future version. If your operation requires utilizing the groupby keys, iterate over the groupby object instead.
  .agg(


,in_history_frac,n_shows_mean,n_unique_pub_mean,mean_cpm_mean,median_cpm_mean,quantile_cpm_10,quantile_cpm_90,roll_cpm_mean,roll_cpm_median,roll_cpm_std
campaign_idx,,,,,,,,,,
5,0.362150,983,12,168.045249,102.670,30.00,335.736,201.997448,118.740,242.926025
10,0.303333,756,13,177.766852,102.410,30.00,372.880,164.446401,90.005,195.548237
22,0.070109,203,5,96.473300,62.000,30.00,200.000,195.177506,110.000,262.842867
23,0.095794,124,4,121.454194,58.765,30.00,285.070,168.123814,94.750,213.902058
26,0.635377,11827,19,182.304097,105.000,30.02,373.168,179.678743,101.250,221.974886


# ЭТАП 8. Агрегации по пользователям в рамках кампании с прогрессом и оценкой времени

In [9]:
import time
from tqdm.notebook import tqdm  # один раз вверху ноутбука

campaign_ids = hist_with_campaign["campaign_idx"].unique()
campaign_ids = np.sort(campaign_ids)

results = []
start_time = time.time()

for i, cid in enumerate(tqdm(campaign_ids, desc="Агрегация по кампаниям")):
    grp = hist_with_campaign[hist_with_campaign["campaign_idx"] == cid]

    res = {
        "campaign_idx": cid,
        "in_history_frac": grp["user_id"].nunique(),  # позже можно поделить на audience_size
        "n_shows_mean": grp["user_id"].count(),
        "n_unique_pub_mean": grp["publisher"].nunique(),
        "mean_cpm_mean": grp["cpm"].mean(),
        "median_cpm_mean": grp["cpm"].median(),
        "quantile_cpm_10": grp["cpm"].quantile(0.10),
        "quantile_cpm_90": grp["cpm"].quantile(0.90),
        "roll_cpm_mean": grp["roll_cpm_mean"].mean(),
        "roll_cpm_median": grp["roll_cpm_median"].median(),
        "roll_cpm_std": grp["roll_cpm_std"].mean(),
    }
    results.append(res)

    # грубая оценка оставшегося времени каждые 50 кампаний
    if (i + 1) % 50 == 0:
        elapsed = time.time() - start_time
        avg_per_campaign = elapsed / (i + 1)
        remaining = avg_per_campaign * (len(campaign_ids) - (i + 1))
        print(
            f"Обработано {i+1}/{len(campaign_ids)} кампаний. "
            f"Примерно осталось: {remaining/60:.1f} минут."
        )

user_agg = pd.DataFrame(results).set_index("campaign_idx")

cache_path = os.path.join(FOLDER, "user_agg.parquet")
user_agg.to_parquet(cache_path)
print("user_agg сохранён в", cache_path)

Агрегация по кампаниям:   0%|          | 0/536 [00:00<?, ?it/s]

Обработано 50/536 кампаний. Примерно осталось: 0.1 минут.
Обработано 100/536 кампаний. Примерно осталось: 0.1 минут.
Обработано 150/536 кампаний. Примерно осталось: 0.1 минут.
Обработано 200/536 кампаний. Примерно осталось: 0.1 минут.
Обработано 250/536 кампаний. Примерно осталось: 0.1 минут.
Обработано 300/536 кампаний. Примерно осталось: 0.0 минут.
Обработано 350/536 кампаний. Примерно осталось: 0.0 минут.
Обработано 400/536 кампаний. Примерно осталось: 0.0 минут.
Обработано 450/536 кампаний. Примерно осталось: 0.0 минут.
Обработано 500/536 кампаний. Примерно осталось: 0.0 минут.
user_agg сохранён в /content/drive/MyDrive/VK_Project_v2/user_agg.parquet


In [10]:
# Сохранение результатов для повторного использования
cache_path = os.path.join(FOLDER, "user_agg.parquet")
user_agg.to_parquet(cache_path)
print("user_agg сохранён в", cache_path)

user_agg сохранён в /content/drive/MyDrive/VK_Project_v2/user_agg.parquet


# ЭТАП 9. Базовые признаки кампании из validate

In [12]:
campaign_basic = validate.copy()

campaign_basic["window_hours"] = campaign_basic["hour_end"] - campaign_basic["hour_start"]
campaign_basic["n_publishers"] = campaign_basic["publishers"].str.split(",").apply(len)

campaign_basic_feats = campaign_basic[["audience_size", "window_hours", "cpm", "n_publishers"]]
campaign_basic_feats.head()

,audience_size,window_hours,cpm,n_publishers
0,1906,95,220.0,2
1,1380,6,312.0,2
2,888,20,70.0,6
3,440,82,240.0,2
4,1476,238,262.0,4


# ЭТАП 10. Объединяем все признаки в features_df

In [13]:
# индексы:
# demography_agg.index — индекс validate
# user_agg.index — индекс validate (через campaign_idx, см. выше)
# campaign_basic_feats.index — индекс validate

features_df = pd.concat(
    [
        demography_agg[["campaign_cluster", "sex_mean", "age_mean", "city_nunique"]],
        user_agg,
        campaign_basic_feats,
    ],
    axis=1,
)

print("features_df.shape:", features_df.shape)
display(features_df.head())

features_df.shape: (1098808, 18)


,campaign_cluster,sex_mean,age_mean,city_nunique,in_history_frac,n_shows_mean,n_unique_pub_mean,mean_cpm_mean,median_cpm_mean,quantile_cpm_10,quantile_cpm_90,roll_cpm_mean,roll_cpm_median,roll_cpm_std,audience_size,window_hours,cpm,n_publishers
0,1,2.0,26.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1906.0,95.0,220.0,2.0
1,1,2.0,33.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1380.0,6.0,312.0,2.0
2,2,2.0,20.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,888.0,20.0,70.0,6.0
3,3,1.0,41.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,440.0,82.0,240.0,2.0
4,1,1.0,36.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1476.0,238.0,262.0,4.0


# Сохраним данные

In [14]:
features_path = os.path.join(FOLDER, "features_df.parquet")
features_df.to_parquet(features_path)
print("features_df сохранён в", features_path)

features_df сохранён в /content/drive/MyDrive/VK_Project_v2/features_df.parquet
